In [9]:
import pandas as pd

visits = pd.read_csv("../data/raw/visits.csv")
regs   = pd.read_csv("../data/raw/regs.csv")

visits.head(), regs.head()

(                                   uuid platform  \
 0  1de9ea66-70d3-4a1f-8735-df5ef7697fb9      web   
 1  f149f542-e935-4870-9734-6b4501eaf614      web   
 2  f149f542-e935-4870-9734-6b4501eaf614      web   
 3  08f0ebd4-950c-4dd9-8e97-b5bdf073eed1      web   
 4  08f0ebd4-950c-4dd9-8e97-b5bdf073eed1      web   
 
                                           user_agent                 date  
 0  Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_2...  2023-03-01T13:29:22  
 1  Mozilla/5.0 (X11; CrOS x86_64 8172.45.0) Apple...  2023-03-01T16:44:28  
 2  Mozilla/5.0 (X11; CrOS x86_64 8172.45.0) Apple...  2023-03-06T06:12:36  
 3  Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:109...  2023-03-01T20:16:37  
 4  Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:109...  2023-03-05T17:42:47  ,
                   date  user_id                    email platform  \
 0  2023-03-01T00:25:39  8838849     joseph95@example.org      web   
 1  2023-03-01T14:53:01  8741065  janetsuarez@example.net      web   
 2  202

In [10]:
# Преобразуем колонки к удобному виду

# переводим названия колонок в нижний регистр (на всякий случай)
visits.columns = [c.lower() for c in visits.columns]
regs.columns = [c.lower() for c in regs.columns]

# даты
visits["date"] = pd.to_datetime(visits["date"], errors="coerce")
regs["date"] = pd.to_datetime(regs["date"], errors="coerce")

# идентификаторы как строки
visits["uuid"] = visits["uuid"].astype(str)
regs["user_id"] = regs["user_id"].astype(str)

# категории к нижнему регистру
for df, cols in [(visits, ["platform"]), (regs, ["platform", "registration_type"])]:
    for c in cols:
        df[c] = df[c].astype(str).str.strip().str.lower()

visits.dtypes, regs.dtypes


(uuid                  object
 platform              object
 user_agent            object
 date          datetime64[ns]
 dtype: object,
 date                 datetime64[ns]
 user_id                      object
 email                        object
 platform                     object
 registration_type            object
 dtype: object)

In [11]:
print("Visits shape:", visits.shape)
print("Registrations shape:", regs.shape)

print("Доля дубликатов в visits:", visits.duplicated().mean())
print("Доля дубликатов в registrations:", regs.duplicated().mean())

Visits shape: (1000, 4)
Registrations shape: (1000, 5)
Доля дубликатов в visits: 0.0
Доля дубликатов в registrations: 0.0


In [12]:
print("Пропуски в visits:")
print(visits.isna().mean())

print("\nПропуски в registrations:")
print(regs.isna().mean())

Пропуски в visits:
uuid          0.0
platform      0.0
user_agent    0.0
date          0.0
dtype: float64

Пропуски в registrations:
date                 0.0
user_id              0.0
email                0.0
platform             0.0
registration_type    0.0
dtype: float64


In [13]:
print("Visits by platform:")
print(visits["platform"].value_counts())

print("\nRegistrations by platform:")
print(regs["platform"].value_counts())

print("\nRegistration types:")
print(regs["registration_type"].value_counts())

Visits by platform:
platform
web        954
android     27
ios         19
Name: count, dtype: int64

Registrations by platform:
platform
android    517
web        265
ios        218
Name: count, dtype: int64

Registration types:
registration_type
email     446
google    303
apple     178
yandex     73
Name: count, dtype: int64


In [14]:
import pandas as pd
import requests

BASE = "https://data-charts-api.hexlet.app"
BEGIN = "2023-03-01"
END   = "2023-09-01"
params = {"begin": BEGIN, "end": END}
params

{'begin': '2023-03-01', 'end': '2023-09-01'}

In [16]:
# 1) делаем GET-запрос
resp_v = requests.get(f"{BASE}/visits", params=params, timeout=60)
resp_v.raise_for_status()  # если что-то не так — будет понятная ошибка

# 2) преобразуем JSON -> DataFrame
visits_api = pd.DataFrame(resp_v.json())

# 3) приводим имена и типы под нашу схему (как в файлах)
# из API: visit_id, platform, user_agent, datetime
# надо:   uuid,     platform, user_agent, date (datetime64)
visits_api = visits_api.rename(columns={"visit_id": "uuid", "datetime": "date"})
visits_api["date"] = pd.to_datetime(visits_api["date"], errors="coerce")
for c in ["uuid", "platform", "user_agent"]:
    visits_api[c] = visits_api[c].astype(str).str.strip().str.lower()

visits_api.head(), visits_api.shape


(                                   uuid platform  \
 0  1de9ea66-70d3-4a1f-8735-df5ef7697fb9      web   
 1  f149f542-e935-4870-9734-6b4501eaf614      web   
 2  08f0ebd4-950c-4dd9-8e97-b5bdf073eed1      web   
 3  19322fed-157c-49c6-b16e-2d5cabeb9592      web   
 4  04762a22-3c9f-40c9-9ac9-6628c4381836      web   
 
                                           user_agent                date  
 0  mozilla/5.0 (windows nt 10.0; win64; x64) appl... 2023-03-01 10:36:22  
 1  mozilla/5.0 (windows nt 10.0; wow64; trident/7... 2023-03-01 06:25:00  
 2  mozilla/5.0 (macintosh; intel mac os x 10_11_2... 2023-03-01 10:26:13  
 3  mozilla/5.0 (macintosh; intel mac os x 10_15_7... 2023-03-01 12:33:06  
 4  mozilla/5.0 (windows nt 10.0; win64; x64) appl... 2023-03-01 01:38:35  ,
 (263459, 4))

In [17]:
resp_r = requests.get(f"{BASE}/registrations", params=params, timeout=60)
resp_r.raise_for_status()

regs_api = pd.DataFrame(resp_r.json())

# из API: user_id, email, platform, registration_type, datetime
# надо:   user_id, email, platform, registration_type, date (datetime64)
regs_api = regs_api.rename(columns={"datetime": "date"})
regs_api["date"] = pd.to_datetime(regs_api["date"], errors="coerce")
for c in ["user_id", "email", "platform", "registration_type"]:
    regs_api[c] = regs_api[c].astype(str).str.strip().str.lower()

regs_api.head(), regs_api.shape

(                 date                               user_id  \
 0 2023-03-01 07:40:13  2e0f6bb8-b029-4f45-a786-2b53990d37f1   
 1 2023-03-01 13:14:00  f007f97c-9d8b-48b5-af08-119bb8f6d9b6   
 2 2023-03-01 03:05:50  24ff46ae-32b3-4a74-8f27-7cf0b8f32f15   
 3 2023-03-01 00:04:47  3e9914e1-5d73-4c23-b25d-b59a3aeb2b60   
 4 2023-03-01 18:31:52  27f875fc-f8ce-4aeb-8722-0ecb283d0760   
 
                         email platform registration_type  
 0           ebyrd@example.org      web            google  
 1    knightgerald@example.org      web             email  
 2  cherylthompson@example.com      web             apple  
 3       halldavid@example.org      web             email  
 4        denise86@example.net      web            google  ,
 (21836, 5))

In [18]:
print("visits_api:", visits_api.shape, 
      "| nulls:", visits_api.isna().mean().round(4).to_dict())
print("regs_api:", regs_api.shape, 
      "| nulls:", regs_api.isna().mean().round(4).to_dict())

print("\nVisits platforms:\n", visits_api["platform"].value_counts().head(10))
print("\nRegs platforms:\n", regs_api["platform"].value_counts().head(10))
print("\nRegs types:\n", regs_api["registration_type"].value_counts().head(10))


visits_api: (263459, 4) | nulls: {'uuid': 0.0, 'platform': 0.0, 'user_agent': 0.0, 'date': 0.0}
regs_api: (21836, 5) | nulls: {'date': 0.0, 'user_id': 0.0, 'email': 0.0, 'platform': 0.0, 'registration_type': 0.0}

Visits platforms:
 platform
web        236301
android     13972
bot          7382
ios          5804
Name: count, dtype: int64

Regs platforms:
 platform
android    10582
web         6877
ios         4377
Name: count, dtype: int64

Regs types:
 registration_type
email     8996
google    7105
apple     4006
yandex    1729
Name: count, dtype: int64


In [19]:
# каталоги под данные (если вдруг нет)
import os
os.makedirs("../data/raw", exist_ok=True)
os.makedirs("../data/processed", exist_ok=True)

# 1) «сырые» ответы API в JSON — на всякий случай
import json
with open(f"../data/raw/visits_{BEGIN}_{END}.json", "w", encoding="utf-8") as f:
    json.dump(resp_v.json(), f, ensure_ascii=False, indent=2)
with open(f"../data/raw/registrations_{BEGIN}_{END}.json", "w", encoding="utf-8") as f:
    json.dump(resp_r.json(), f, ensure_ascii=False, indent=2)

# 2) «чистые» CSV для дальнейшей работы
visits_api.to_csv("../data/processed/visits_api_clean.csv", index=False)
regs_api.to_csv("../data/processed/registrations_api_clean.csv", index=False)

"Saved!"

'Saved!'

In [20]:
v_local = pd.read_csv("../data/raw/visits.csv")
r_local = pd.read_csv("../data/raw/regs.csv")

v_local["date"] = pd.to_datetime(v_local["date"], errors="coerce")
r_local["date"] = pd.to_datetime(r_local["date"], errors="coerce")

print("local visits date range:", v_local["date"].min(), "→", v_local["date"].max())
print("api   visits date range:", visits_api["date"].min(), "→", visits_api["date"].max())

print("local regs   date range:", r_local["date"].min(), "→", r_local["date"].max())
print("api   regs   date range:", regs_api["date"].min(), "→", regs_api["date"].max())

local visits date range: 2023-03-01 00:05:35 → 2023-03-07 23:05:08
api   visits date range: 2023-03-01 00:00:43 → 2023-08-31 23:52:57
local regs   date range: 2023-03-01 00:12:22 → 2023-03-05 22:04:01
api   regs   date range: 2023-03-01 00:04:47 → 2023-08-31 23:43:26


In [21]:
# убираем визиты ботов
visits_clean = visits_api[~visits_api["user_agent"].str.contains("bot", case=False, na=False)].copy()

# для каждого uuid оставим только последний визит
visits_clean = visits_clean.sort_values("date").drop_duplicates("uuid", keep="last")

# агрегируем: количество визитов по дате и платформе
visits_grouped = visits_clean.groupby([visits_clean["date"].dt.date, "platform"]).size().reset_index(name="visits")

visits_grouped = visits_grouped.rename(columns={"date": "date_group"})
visits_grouped.head()

,date_group,platform,visits
0,2023-03-01,android,75
1,2023-03-01,ios,22
2,2023-03-01,web,279
3,2023-03-02,android,67
4,2023-03-02,ios,31


In [22]:
# агрегируем регистрации: количество по дате и платформе
regs_grouped = regs_api.groupby([regs_api["date"].dt.date, "platform"]).size().reset_index(name="registrations")

regs_grouped = regs_grouped.rename(columns={"date": "date_group"})
regs_grouped.head()

,date_group,platform,registrations
0,2023-03-01,android,61
1,2023-03-01,ios,18
2,2023-03-01,web,8
3,2023-03-02,android,59
4,2023-03-02,ios,24


In [23]:
# объединение по дате и платформе
merged = pd.merge(visits_grouped, regs_grouped, on=["date_group", "platform"], how="outer")

# если где-то нет регистраций или визитов → ставим 0
merged = merged.fillna(0)

# конверсия = (registrations / visits * 100)
merged["conversion"] = (merged["registrations"] / merged["visits"] * 100).round(2)

# сортируем по дате
merged = merged.sort_values("date_group").reset_index(drop=True)

merged.head(10)


,date_group,platform,visits,registrations,conversion
0,2023-03-01,android,75,61,81.33
1,2023-03-01,ios,22,18,81.82
2,2023-03-01,web,279,8,2.87
3,2023-03-02,android,67,59,88.06
4,2023-03-02,ios,31,24,77.42
5,2023-03-02,web,515,23,4.47
6,2023-03-03,android,26,22,84.62
7,2023-03-03,ios,40,34,85.00
8,2023-03-03,web,617,51,8.27
9,2023-03-04,android,94,77,81.91


In [25]:
merged.to_json("../data/processed/conversion.json")

In [33]:
# 1) читаем файл рекламы
# если вдруг у тебя разделитель ; (точка с запятой), смени на: pd.read_csv("../data/raw/ads.csv", sep=";")
ads = pd.read_csv("../data/raw/ads.csv")

# 2) приведение типов/колонок
# ожидаемые колонки: date, utm_source, utm_medium, utm_campaign, cost
ads.columns = [c.strip().lower() for c in ads.columns]
ads["date"] = pd.to_datetime(ads["date"], errors="coerce")
ads["date_group"] = ads["date"].dt.date

# 3) оставим только нужные поля (для задачи нам нужны дата, кампания, затраты)
ads = ads[["date_group", "utm_campaign", "cost"]]

ads.head()

,date_group,utm_campaign,cost
0,2023-03-01,advanced_algorithms_series,212
1,2023-03-02,advanced_algorithms_series,252
2,2023-03-03,advanced_algorithms_series,202
3,2023-03-04,advanced_algorithms_series,223
4,2023-03-05,advanced_algorithms_series,265


In [34]:
ads_grouped = (
    ads.groupby(["date_group", "utm_campaign"], as_index=False)["cost"]
       .sum()
)
ads_grouped.head()


,date_group,utm_campaign,cost
0,2023-03-01,advanced_algorithms_series,212
1,2023-03-02,advanced_algorithms_series,252
2,2023-03-03,advanced_algorithms_series,202
3,2023-03-04,advanced_algorithms_series,223
4,2023-03-05,advanced_algorithms_series,265


In [35]:
# если у тебя уже есть merged из предыдущего шага
conv_daily = merged.groupby("date_group", as_index=False)[["visits", "registrations"]].sum()
conv_daily.head()

,date_group,visits,registrations
0,2023-03-01,376,87
1,2023-03-02,613,106
2,2023-03-03,683,107
3,2023-03-04,647,159
4,2023-03-05,707,115


In [36]:
final = conv_daily.merge(ads_grouped, on="date_group", how="left")

# если рекламы в этот день не было — ставим «none» и 0
final["utm_campaign"] = final["utm_campaign"].fillna("none")
final["cost"] = final["cost"].fillna(0)

# сортируем по дате
final = final.sort_values("date_group").reset_index(drop=True)
final.head(10)

,date_group,visits,registrations,utm_campaign,cost
0,2023-03-01,376,87,advanced_algorithms_series,212.0
1,2023-03-02,613,106,advanced_algorithms_series,252.0
2,2023-03-03,683,107,advanced_algorithms_series,202.0
3,2023-03-04,647,159,advanced_algorithms_series,223.0
4,2023-03-05,707,115,advanced_algorithms_series,265.0
5,2023-03-06,1291,230,advanced_algorithms_series,108.0
6,2023-03-07,1382,124,advanced_algorithms_series,165.0
7,2023-03-08,1382,151,advanced_algorithms_series,155.0
8,2023-03-09,1064,209,advanced_algorithms_series,124.0
9,2023-03-10,812,112,advanced_algorithms_series,276.0


In [37]:
final.to_json("../data/processed/ads.json")
"saved: ../data/processed/ads.json"

'saved: ../data/processed/ads.json'